# **☆ AI Zebrafish Project ☆**

### NOTE. Code Adjustment

1. 將 valid set 改為和 test set 有預切 + 沒有 aug
2. `drop_too_dark()` 計算 < intensity 的數量同時將 > intensity 的 pixel 歸零

### ***Common Settings and Components***

Dataset Version Note

v0-2, Half_AP_Size
- {Dataset}_Cropped_v0: RGB_HE_fusion, StdevCluster, `name_example: L_fish_111_A_selected_4.tiff` ( 沒有特別標註 U, D 分配到哪 )
- {Dataset}_Cropped_v1: 開始應用不同 result (AliasInPath) + KMeansCluster, `name_example: L_fish_111_A_selected_4.tiff` ( 沒有特別標註 U, D 分配到哪 )
- {Dataset}_Cropped_v2: 改採 xlsx 紀錄資訊 (1_Cropset_N_XLSX), `name_example: fish_100_A_U_crop_30.tiff`

v3, 修正 label leakage
- {Dataset}_Cropped_v3: Original_AP_Size, `name_example: fish_100_A_crop_30.tiff`
- {Dataset}_Cropped_v3_HF: Half_AP_Size, `name_example: fish_100_A_U_crop_30.tiff`

Storage and `Fiji (ImageJ)` Location Config : [db_path_plan.toml](script/db_path_plan.toml)

In [ ]:
from modules.data.ij.zfij import ZFIJ
from modules.data.processeddatainstance import ProcessedDataInstance
from modules.shared.utils import get_repo_root
# -----------------------------------------------------------------------------/

""" Detect Repository """
print(f"Repository: '{get_repo_root()}'")

Zebrafish_IJ = ZFIJ()
Processed_DI = ProcessedDataInstance()

### ***STEP 0. process_data***

##### To adjust

palmskin
* 0.2.1.preprocess_palmskin.py
* 0.2.2.select_palmskin_manualroi.py # 選ROI (`ManualROI.roi`) ----- **拆分 0.2.3.process_palmskin_manualroi.py**
* 0.2.3.apply_palmskin_manualroi.py # apply on one of results ----- **拆分 0.2.3.process_palmskin_manualroi.py**
* 0.2.4.process_palmskin_fixroi.py

brightfield
* 0.3.1.analyze_brightfield.py
* 0.3.2.process_brightfield_manualarea.py # 選ROI + cvt_mask + 測量 (`02_cropped_BF.tif`)
* 0.3.3.train_brightfield_unetarea.py ----- **ZebraFish_BF_Seg**
* 0.3.4.predict_brightfield_unetarea.py ----- **ZebraFish_BF_Seg**
* 0.3.5.measure_unet_area.py ----- **重命名 0.3.2.measure_unet_area.py**

##### 0.1.x. name check

[0.1.1.check_lif_name.py][script]
( [0.1.1.check_lif_name.toml][Config] )

[script]: script/0.process_data/0.1.1.check_lif_name.py
[Config]: script/Config/0.1.1.check_lif_name.toml

In [ ]:
from modules.data.lif.batchlifnamechecker import BatchLIFNameChecker
# -----------------------------------------------------------------------------/

batch_lif_name_checker = BatchLIFNameChecker(zfij_instance=Zebrafish_IJ)
batch_lif_name_checker.run("0.1.1.check_lif_name.toml")

##### 0.2.x. palmskin

[0.2.1. preprocess_palmskin.py][script]
( [0.2.1.preprocess_palmskin.toml][Config] )

[script]: script/0.process_data/0.2.1.preprocess_palmskin.py
[Config]: script/Config/0.2.1.preprocess_palmskin.toml

In [ ]:
from modules.data.lif.palmskinpreprocesser import PalmskinPreprocesser
# -----------------------------------------------------------------------------/

palmskin_preprocesser = PalmskinPreprocesser(zfij_instance=Zebrafish_IJ)
palmskin_preprocesser.run("0.2.1.preprocess_palmskin.toml")

[0.2.3.process_palmskin_manualroi.py][script]
( [0.2.3.process_palmskin_manualroi.toml][Config] )

[script]: script/0.process_data/0.2.3.process_palmskin_manualroi.py
[Config]: script/Config/0.2.3.process_palmskin_manualroi.toml

In [ ]:
from modules.data.lif.palmskinmanualroicreator import PalmskinManualROICreator
# -----------------------------------------------------------------------------/

palmskin_manualroi_creator = PalmskinManualROICreator(zfij_instance=Zebrafish_IJ,
                                                      processed_data_instance=Processed_DI)
palmskin_manualroi_creator.run("0.2.3.process_palmskin_manualroi.toml")

[0.2.4.process_palmskin_fixroi.py][script]
( [0.2.3.process_palmskin_manualroi.toml][Config] )

[script]: script/0.process_data/0.2.4.process_palmskin_fixroi.py
[Config]: script/Config/0.2.3.process_palmskin_manualroi.toml

In [ ]:
from modules.data.lif.palmskinfixedroicreator import PalmskinFixedROICreator
# -----------------------------------------------------------------------------/

palmskin_fixedroi_creator = PalmskinFixedROICreator(processed_data_instance=Processed_DI)
palmskin_fixedroi_creator.run("0.2.3.process_palmskin_manualroi.toml")

##### 0.3.x. brightfield

[0.3.1.analyze_brightfield.py][script]
( [0.3.1.analyze_brightfield.toml][Config] )

[script]: script/0.process_data/0.3.1.analyze_brightfield.py
[Config]: script/Config/0.3.1.analyze_brightfield.toml

In [ ]:
from modules.data.lif.brightfieldanalyzer import BrightfieldAnalyzer
# -----------------------------------------------------------------------------/

brightfield_analyzer = BrightfieldAnalyzer(zfij_instance=Zebrafish_IJ)
brightfield_analyzer.run("0.3.1.analyze_brightfield.toml")

[0.3.2.measure_unet_area.py][script]
( [0.3.1.analyze_brightfield.toml][Config] )

[script]: script/0.process_data/0.3.2.measure_unet_area.py
[Config]: script/Config/0.3.1.analyze_brightfield.toml

In [ ]:
from modules.data.lif.brightfieldunetareameter import BrightfieldUNetAreaMeter
# -----------------------------------------------------------------------------/

brightfield_unet_area_meter = BrightfieldUNetAreaMeter(zfij_instance=Zebrafish_IJ,
                                                       processed_data_instance=Processed_DI)
brightfield_unet_area_meter.run("0.3.1.analyze_brightfield.toml")

##### 0.4.x. `collect` and `copy` results

[0.4.1.collect_results.py][script]
( [0.4.1.collect_results.toml][Config] )

[script]: script/0.process_data/0.4.1.collect_results.py
[Config]: script/Config/0.4.1.collect_results.toml

In [ ]:
Processed_DI.collect_results("0.4.collect_results.toml")

[0.4.2.copy_results.py][script]
( [0.4.2.copy_results.toml][Config] )

[script]: script/0.process_data/0.4.2.copy_results.py
[Config]: script/Config/0.4.2.copy_results.toml

In [ ]:
# TODO:  Not a component yet, run the script directly

##### 0.5.x. `split` and `cluster` data

[0.5.1.create_tabular_file.py][script]
( [0.5.cluster_data.toml][Config] )

[script]: script/0.process_data/0.5.1.create_tabular_file.py
[Config]: script/Config/0.5.cluster_data.toml

In [ ]:
Processed_DI.create_tabular_file("0.5.cluster_data.toml")

[0.5.2.split_data.py][script]
( [0.5.cluster_data.toml][Config] )

[script]: script/0.process_data/0.5.2.split_data.py
[Config]: script/Config/0.5.cluster_data.toml

In [ ]:
# TODO:  Not a component yet, run the script directly

[0.5.3.cluster_data.py][script]
( [0.5.cluster_data.toml][Config] )

[script]: script/0.process_data/0.5.3.cluster_data.py
[Config]: script/Config/0.5.cluster_data.toml

In [ ]:
from modules.data.clustering.surfaceareakmeanscluster import SurfaceAreaKMeansCluster
# -----------------------------------------------------------------------------/

sa_kmeans_cluster = SurfaceAreaKMeansCluster(processed_data_instance=Processed_DI)
sa_kmeans_cluster.run("0.5.cluster_data.toml")

[0.5.4.plot_cluster.py][script]
( [0.5.cluster_data.toml][Config] )

[script]: script/0.process_data/0.5.4.plot_cluster.py
[Config]: script/Config/0.5.cluster_data.toml

In [ ]:
from modules.plot.clustering.surfaceareakmeansplotter import SurfaceAreaKMeansPlotter
# -----------------------------------------------------------------------------/

sa_kmeans_plotter = SurfaceAreaKMeansPlotter(processed_data_instance=Processed_DI)
sa_kmeans_plotter.run("0.5.cluster_data.toml")

### ***STEP 1. make_dataset***

[1.1.crop_images.py][script]
( [1.make_dataset.toml][Config] )

[script]: script/1.make_dataset/1.1.crop_images.py
[Config]: script/Config/1.make_dataset.toml

In [ ]:
from modules.data.dataset.imagecropper import ImageCropper
# -----------------------------------------------------------------------------/

image_cropper = ImageCropper(processed_data_instance=Processed_DI)
image_cropper.run("1.make_dataset.toml")

[1.2.create_dataset_file.py][script]
( [1.make_dataset.toml][Config] )

[script]: script/1.make_dataset/1.2.create_dataset_file.py
[Config]: script/Config/1.make_dataset.toml

In [ ]:
from modules.data.dataset.datasetfilecreator import DatasetFileCreator
# -----------------------------------------------------------------------------/

dataset_file_creator = DatasetFileCreator(processed_data_instance=Processed_DI)
dataset_file_creator.run("1.make_dataset.toml")

### ***STEP 2. training***

[2.a.vit_b_16.py][script]
( [2.training.toml][Config] )

[script]: script/2.training/2.a.vit_b_16.py
[Config]: script/Config/2.training.toml

In [ ]:
# With batch config mode, run the script directly

### ***STEP 3. test_by_img***

[3.a.vit_b_16.py][script]
( [3.test_by_image.toml][Config] )

[script]: script/3.test_by_img/3.a.vit_b_16.py
[Config]: script/Config/3.test_by_image.toml

In [ ]:
from modules.dl.tester.imagetester.vitb16imagetester import VitB16ImageTester
# -----------------------------------------------------------------------------/

vit_b_16_image_tester = VitB16ImageTester()
vit_b_16_image_tester.run("3.test_by_image.toml")

### ***STEP 4. test_by_fish***

[4.a.vit_b_16.py][script]
( [4.test_by_fish.toml][Config] )

[script]: script/4.test_by_fish/4.a.vit_b_16.py
[Config]: script/Config/4.test_by_fish.toml

In [ ]:
from modules.dl.tester.fishtester.vitb16fishtester import VitB16FishTester
# -----------------------------------------------------------------------------/

vit_b_16_fish_tester = VitB16FishTester()
vit_b_16_fish_tester.run("4.test_by_fish.toml")

### ***STEP 5. make_cam_gallery***

[5.a.mk_cam_gallery.py][script]
( [5.make_cam_gallery.toml][Config] )

[script]: script/5.make_cam_gallery/5.a.mk_cam_gallery.py
[Config]: script/Config/5.make_cam_gallery.toml

In [ ]:
from modules.plot.cam_gallery.creator.camgallerycreator import CamGalleryCreator
# -----------------------------------------------------------------------------/

cam_gallery_creator = CamGalleryCreator()
cam_gallery_creator.run("5.make_cam_gallery.toml")

[5.b.mt_cam_gallery.py][script]
( [5.make_cam_gallery.toml][Config] )

[script]: script/5.make_cam_gallery/5.b.mt_cam_gallery.py
[Config]: script/Config/5.make_cam_gallery.toml

In [ ]:
# Enhance by multi-threading, run the script directly